In [1]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import pandas as pd
from tqdm import tqdm 

In [2]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [3]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("Ternary_model_state_dict_learning_New_epoch_15.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("Ternary_optimizer_state_learning_dict_New_epoch_15.pt")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense

In [4]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 모든 Dropout 레이어의 비율 변경
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = new_dropout_rate


In [ ]:
# 모델 구조 확인
print(model)

In [ ]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 실제로 예측해보기

In [6]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [7]:
df = pd.read_csv("../dataset/2022산림복지_본문_맞춤법검사.txt", sep="\t", encoding='UTF-8')
df

,split_str,org_idx,correct_str
0,치유나무숲아웃도어연구회 산림치유지도사1급 산림치유 미래유망자격 산림치유지도사2급 산...,0,치유 나무숲 야외 연구회 산림치유지도사 1급 산림치유 미래 유망자격 산림치유지도사 ...
1,치유나무숲 회원님들 중에서도 이번 시험에 응시하시는 몇 분이 계십니다,0,치유 나무숲 회원님들 중에서도 이번 시험에 응시하시는 몇 분이 계십니다
2,회원님들 응원과 격려를 위해 그리고 치유나무숲아웃도어연구회 홍보 그리고 1급 기출문...,0,회원님들 응원과 격려를 위해 그리고 치유 나무숲 야외 연구회 홍보 그리고 1급 기출...
3,2024년 2월 평가시험에 응시하시는 수험생 및 관련 선생님들은 잠시 치유나무숲 부...,0,2024년 2월 평가시험에 응시하시는 수험생 및 관련 선생님들은 잠시 치유 나무숲 ...
4,출처 https license fowi or kr board selectNotice...,0,출처 https license or kr board select noticeboa...
...,...,...,...
262303,예를 들어 현재 전북 전남 경남 3개 도에 걸쳐 있는 지리산 1915m 부근을 비우...,9974,"예를 들어 현재 전북 전남 경남 3개 도에 걸쳐 있는 지리산 1,915m 부근을 비..."
262304,지리산이 걸쳐 있는 8개 행정구역 가운데 진주를 제외하면 모두 인구 소멸 우려지역이...,9974,지리산이 걸쳐 있는 8개 행정구역 가운데 진주를 제외하면 모두 인구 소멸 우려지역이...
262305,지역을 비워두는 것도 한 전략이라는 말이다 그리 되면 캘리포니아 시에라네바다산맥 너...,9974,지역을 비워두는 것도 한 전략이라는 말이다. 그리되면 캘리포니아 시에라네바다산맥 너...
262306,문제는 내 지역구 소멸 을 반대하는 지역 정치인들과 토호세력들인데 국민 저변의 사회...,9974,문제는 내 지역구 소멸을 반대하는 지역 정치인들과 토호 세력들인데 국민 저변의 사회...


In [8]:
# 한국어 문장 입력 받기
korean_sentences = df['correct_str'].tolist()

# 예측 라벨 출력
emotion_labels = {0:'부정', 1:'중립', 2:'긍정'}
predicted_label = [emotion_labels[predict_label(korean_sentence, model, tokenizer, device)] for korean_sentence in tqdm(korean_sentences)]
df['label'] = predicted_label

100%|██████████| 262308/262308 [2:44:51<00:00, 26.52it/s]  


In [9]:
ddff = df[['correct_str','label','org_idx']]
ddff

,correct_str,label,org_idx
0,치유 나무숲 야외 연구회 산림치유지도사 1급 산림치유 미래 유망자격 산림치유지도사 ...,긍정,0
1,치유 나무숲 회원님들 중에서도 이번 시험에 응시하시는 몇 분이 계십니다,부정,0
2,회원님들 응원과 격려를 위해 그리고 치유 나무숲 야외 연구회 홍보 그리고 1급 기출...,긍정,0
3,2024년 2월 평가시험에 응시하시는 수험생 및 관련 선생님들은 잠시 치유 나무숲 ...,긍정,0
4,출처 https license or kr board select noticeboa...,부정,0
...,...,...,...
262303,"예를 들어 현재 전북 전남 경남 3개 도에 걸쳐 있는 지리산 1,915m 부근을 비...",긍정,9974
262304,지리산이 걸쳐 있는 8개 행정구역 가운데 진주를 제외하면 모두 인구 소멸 우려지역이...,부정,9974
262305,지역을 비워두는 것도 한 전략이라는 말이다. 그리되면 캘리포니아 시에라네바다산맥 너...,긍정,9974
262306,문제는 내 지역구 소멸을 반대하는 지역 정치인들과 토호 세력들인데 국민 저변의 사회...,부정,9974


In [10]:
ddff['label'].value_counts()

긍정    211903
부정     50238
중립       167
Name: label, dtype: int64

In [11]:
ddff.rename(columns={'correct_str': 'sentence'}, inplace=True)
ddff.rename(columns={'label': 'predicted_sentiment'}, inplace=True)
ddff.to_csv("./Ternary_classification_2022.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
